In [ ]:
# Agentic workflow with multiple validated tools (strict tool use)
# Use case: Customer Support Automation

!pip install anthropic --upgrade

import anthropic
from getpass import getpass
import json

# 1. Set up the client
api_key = getpass("Enter your Anthropic API key: ")
client = anthropic.Anthropic(api_key=api_key)

# 2. Define strict tools
tools = [
    {
        "name": "classify_issue",
        "strict": True,
        "input_schema": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "enum": ["billing", "technical", "account", "other"]
                }
            },
            "required": ["category"],
            "additionalProperties": False
        }
    },
    {
        "name": "generate_ticket_fields",
        "strict": True,
        "input_schema": {
            "type": "object",
            "properties": {
                "priority": {
                    "type": "string",
                    "enum": ["low", "medium", "high"]
                },
                "requires_handoff": {"type": "boolean"},
                "tags": {
                    "type": "array",
                    "items": {"type": "string"}
                }
            },
            "required": ["priority", "requires_handoff"],
            "additionalProperties": False
        }
    }
]

# 3. User request (Claude must choose and call tools)
user_request = """
Customer says: "My internet is down since morning. Router lights keep blinking and I tried restarting twice."
"""

response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    betas=["structured-outputs-2025-11-13"],
    max_tokens=1024,
    messages=[{"role": "user", "content": user_request}],
    tools=tools
)

print("Raw content blocks returned by Claude:\n")
for block in response.content:
    print(block.type)
    print(block, "\n")

# 4. Extract tool calls and show schema-validated parameters
tool_calls = [block for block in response.content if block.type == "tool_use"]

print("\nParsed tool calls with strictly validated inputs:\n")
for call in tool_calls:
    print(f"Tool name: {call.name}")
    print("Parameters:")
    print(json.dumps(call.input, indent=2))
    print("-" * 40)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.8 MB/s eta 0:00:00
Enter your Anthropic API key: ··········
Raw content blocks returned by Claude:

text
BetaTextBlock(citations=None, text="I'll help classify this issue and generate the appropriate ticket fields.", type='text') 

tool_use
BetaToolUseBlock(id='toolu_01G72j6kzbwy5fuxQM9EHs5G', input={'category': 'technical'}, name='classify_issue', type='tool_use') 

tool_use
BetaToolUseBlock(id='toolu_01R75abUgA7dvcdoxtkpEesj', input={'priority': 'high', 'requires_handoff': True, 'tags': ['internet_outage', 'connectivity', 'router_issue', 'troubleshooting_attempted']}, name='generate_ticket_fields', type='tool_use') 


Parsed tool calls with strictly validated inputs:

Tool name: classify_issue
Parameters:
{
  "category": "technical"
}
----------------------------------------
Tool name: generate_ticket_fields
Parameters:
{
  "priority": "high",
  "requires_handoff": true,
  "tags": [
    "internet_outage",
    "connectivity"

In [ ]:
# Follow-up: Send tool results back to Claude and let it generate a final summary

import json

# 1. Create fake results from the tools as if your backend executed them
tool_results = []

for call in tool_calls:
    if call.name == "classify_issue":
        # content must be a string or a list of content blocks
        tool_results.append({
            "type": "tool_result",
            "tool_use_id": call.id,
            "content": json.dumps({
                "message": "Issue classified successfully.",
                "category": call.input["category"]
            })
        })

    elif call.name == "generate_ticket_fields":
        tool_results.append({
            "type": "tool_result",
            "tool_use_id": call.id,
            "content": json.dumps({
                "message": "Ticket fields prepared.",
                "priority": call.input["priority"],
                "requires_handoff": call.input["requires_handoff"],
                "tags": call.input["tags"]
            })
        })

# 2. Rebuild the full conversation history manually
messages_history = [
    # Original user request
    {"role": "user", "content": user_request},

    # Assistant turn that included text + tool_use blocks
    {"role": "assistant", "content": response.content},

    # User providing tool results back to Claude (as content blocks)
    {"role": "user", "content": tool_results},

    # New user instruction asking for a summary
    {"role": "user", "content": "Summarize the ticket for the support team."}
]

# 3. Ask Claude to produce the final agent-friendly summary
final_response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    betas=["structured-outputs-2025-11-13"],
    max_tokens=1024,
    messages=messages_history
)

print("\n=== Final Agent Summary from Claude ===\n")
for block in final_response.content:
    if block.type == "text":
        print(block.text)



=== Final Agent Summary from Claude ===

**Ticket Summary for Support Team:**

**Issue:** Complete internet outage since morning

**Key Details:**
- Customer has no internet connectivity
- Router lights are blinking abnormally
- Customer has already attempted basic troubleshooting (restarted router twice with no success)
- Issue duration: Several hours (since morning)

**Classification:** Technical Issue - High Priority

**Recommended Action:** Requires handoff to technical support team for advanced troubleshooting. May need to check:
- ISP service status in customer's area
- Router hardware diagnostics
- Connection between modem and ISP
- Possible equipment replacement if hardware failure

**Tags:** internet_outage, connectivity, router_issue, troubleshooting_attempted


In [ ]:
# Follow-up: Send tool results back and ask Claude for a JSON summary
import json

# 1. Create fake results from the tools as if your backend executed them
tool_results = []

for call in tool_calls:
    if call.name == "classify_issue":
        tool_results.append({
            "type": "tool_result",
            "tool_use_id": call.id,
            # content must be a string or list of content blocks
            "content": json.dumps({
                "message": "Issue classified successfully.",
                "category": call.input["category"]
            })
        })

    elif call.name == "generate_ticket_fields":
        tool_results.append({
            "type": "tool_result",
            "tool_use_id": call.id,
            "content": json.dumps({
                "message": "Ticket fields prepared.",
                "priority": call.input["priority"],
                "requires_handoff": call.input["requires_handoff"],
                "tags": call.input["tags"]
            })
        })

# 2. Define the JSON schema for the final ticket summary
summary_schema = {
    "type": "object",
    "properties": {
        "issue_summary": {"type": "string"},
        "category": {"type": "string", "enum": ["billing", "technical", "account", "other"]},
        "priority": {"type": "string", "enum": ["low", "medium", "high"]},
        "requires_handoff": {"type": "boolean"},
        "tags": {
            "type": "array",
            "items": {"type": "string"}
        },
        "suggested_actions": {
            "type": "array",
            "items": {"type": "string"}
        }
    },
    "required": ["issue_summary", "category", "priority", "requires_handoff"],
    "additionalProperties": False
}

# 3. Rebuild the conversation history
messages_history = [
    {"role": "user", "content": user_request},
    {"role": "assistant", "content": response.content},
    {"role": "user", "content": tool_results},
    {
        "role": "user",
        "content": "Using the tool results, summarize this ticket as JSON only, matching the provided schema."
    }
]

# 4. Ask Claude for a structured JSON summary
final_json_response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    betas=["structured-outputs-2025-11-13"],
    max_tokens=512,
    messages=messages_history,
    output_format={"type": "json_schema", "schema": summary_schema}
)

print("\n=== Raw JSON text from Claude ===\n")
print(final_json_response.content[0].text)

print("\n=== Parsed JSON summary ===\n")
ticket_summary = json.loads(final_json_response.content[0].text)
print(json.dumps(ticket_summary, indent=2))



=== Raw JSON text from Claude ===

{"issue_summary":"Customer experiencing internet outage since morning with router lights blinking. Customer has attempted troubleshooting by restarting router twice without resolution.","category":"technical","priority":"high","requires_handoff":true,"tags":["internet_outage","connectivity","router_issue","troubleshooting_attempted"],"suggested_actions":["Check service status in customer's area","Verify router connection and cable integrity","Run remote diagnostics on router","Schedule technician visit if hardware fault detected","Escalate to network engineering team"]}

=== Parsed JSON summary ===

{
  "issue_summary": "Customer experiencing internet outage since morning with router lights blinking. Customer has attempted troubleshooting by restarting router twice without resolution.",
  "category": "technical",
  "priority": "high",
  "requires_handoff": true,
  "tags": [
    "internet_outage",
    "connectivity",
    "router_issue",
    "troublesh